#  MindMate:AI – An Emotion-Aware Motivational Chatbot
MindMate is a GenAI-powered chatbot designed to provide emotional support, motivation, and companionship. It detects the user's emotions from text or audio and responds with empathy, encouragement, and positivity.

This project uses:
- Google Gemini 1.5 Flash for generating responses
- Emotion detection with keyword mapping
- gTTS (Text-to-Speech) for voice output
- Widgets-based UI with support for personality mode selection and audio uploads

> Built as part of the GenAI Capstone to showcase real-world impact using multi-modal interaction.


In [2]:
# Install necessary libraries
# Core Gemini AI
!pip install -q google-generativeai

# Voice Input and Output
!pip install -q gTTS SpeechRecognition

# UI Elements for Notebooks
!pip install -q ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 51.0 MB/s eta 0:00:00:00:0100:01


In [3]:
# Import required libraries
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Fetch API key stored in Kaggle secrets
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Configure Gemini AI with the fetched API key
genai.configure(api_key=GOOGLE_API_KEY)

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Load Gemini API key securely
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


In [4]:
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash-001",
    system_instruction=(
        "You are a kind, friendly, and supportive AI companion. "
        "Your goal is to provide emotional support, mental wellness advice, and daily motivation. "
        "Keep responses positive, warm, and encouraging. Be empathetic and never judgmental."
    )
)

# Start chat instance
chat = model.start_chat(history=[])



In [5]:
def detect_emotion(text):
    text = text.lower()
    sad = ["sad", "depressed", "tired", "hopeless", "lonely", "cry"]
    angry = ["angry", "mad", "furious", "annoyed"]
    happy = ["happy", "joy", "great", "awesome", "fantastic"]
    
    if any(word in text for word in sad):
        return "sad"
    elif any(word in text for word in angry):
        return "angry"
    elif any(word in text for word in happy):
        return "happy"
    return "neutral"



In [6]:
from gtts import gTTS
from IPython.display import Audio, display
import tempfile

def speak(text):
    tts = gTTS(text)
    with tempfile.NamedTemporaryFile(delete=True, suffix=".mp3") as fp:
        tts.save(fp.name)
        display(Audio(fp.name, autoplay=True))



In [7]:
import speech_recognition as sr

def transcribe_audio(file_bytes):
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
        temp_audio.write(file_bytes)
        temp_audio_path = temp_audio.name

    recognizer = sr.Recognizer()
    with sr.AudioFile(temp_audio_path) as source:
        audio = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return "[Unclear audio]"
        except sr.RequestError:
            return "[API error]"


In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# UI Elements
chat_display = widgets.HTML(value="<b>MindMate is here for you!</b><br><br>")
input_box = widgets.Text(placeholder="Type your message...", description="You:")
send_button = widgets.Button(description="Send 💬", button_style="success")
upload_audio = widgets.FileUpload(accept='.wav', multiple=False)
output = widgets.Output()

# Visual message styling
def show_message(who, text, color):
    return f"<b style='color:{color}'>{who}:</b> {text}<br><br>"

# Handle sending text
def handle_send(_):
    user_text = input_box.value.strip()
    if user_text == "":
        return

    emotion = detect_emotion(user_text)
    prompt = f"[Emotion: {emotion}]\nUser said: {user_text}"

    chat_display.value += show_message("You", user_text, "#0066cc")
    response = chat.send_message(prompt)
    bot_reply = response.text
    chat_display.value += show_message("MindMate", bot_reply, "#009933")

    speak(bot_reply)
    input_box.value = ""

# Handle audio input
def handle_audio_upload(change):
    for name, file_info in upload_audio.value.items():
        transcribed = transcribe_audio(file_info['content'])
        input_box.value = transcribed
        with output:
            print(f"Transcribed: {transcribed}")


In [9]:
modes = {
    "Default": "You are a helpful and supportive chatbot.",
    "Therapist": "You are a calm, empathetic therapist who offers emotional support.",
    "Motivational Coach": "You are a high-energy coach who inspires action.",
    "Friendly Buddy": "You are a cheerful, funny friend who makes people smile.",
    "Wise Mentor": "You are a philosophical and wise mentor with deep advice."
}

mode_selector = widgets.Dropdown(
    options=list(modes.keys()),
    value="Default",
    description="Choose Mode:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)



In [10]:
send_button.on_click(handle_send)
upload_audio.observe(handle_audio_upload, names='value')

display(chat_display)
display(widgets.HBox([input_box, send_button]))
display(widgets.Label("🎤 Upload Audio (WAV):"))
display(upload_audio)
display(mode_selector)
display(output)


HTML(value='<b>MindMate is here for you!</b><br><br>')

Label(value='🎤 Upload Audio (WAV):')

FileUpload(value=(), accept='.wav', description='Upload')

Dropdown(description='Choose Mode:', layout=Layout(width='50%'), options=('Default', 'Therapist', 'Motivationa…

Output()